In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random as rd

In [8]:
input_df = pd.read_csv("pizza_customers.csv")
input_df

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


In [9]:
input_df.columns

Index(['CustomerID', 'Gender', 'Age', 'Annual Income (k$)',
       'Spending Score (1-100)'],
      dtype='object')

In [10]:
corr_dataset = input_df.corr()
corr_dataset.style.background_gradient(cmap='coolwarm').set_precision(2)

,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
CustomerID,1.00,-0.03,0.98,0.01
Age,-0.03,1.00,-0.01,-0.33
Annual Income (k$),0.98,-0.01,1.00,0.01
Spending Score (1-100),0.01,-0.33,0.01,1.00


In [11]:
features = ['Male', 'Age', 'Annual Income (k$)',
       'Spending Score (1-100)']
dummies = pd.get_dummies(input_df['Gender'], drop_first= True)
features_df = pd.concat([input_df, dummies], axis=1)
features_df[features]

,Male,Age,Annual Income (k$),Spending Score (1-100)
0,1,19,15,39
1,1,21,15,81
2,0,20,16,6
3,0,23,16,77
4,0,31,17,40
...,...,...,...,...
195,0,35,120,79
196,0,45,126,28
197,1,32,126,74
198,1,32,137,18


In [61]:
def Kmeans_custom(df_ds,K, no_of_iterations):
  df_ds_c = df_ds[features].values
  size =[]

  m=df_ds_c.shape[0] #number of training examples
  n=df_ds_c.shape[1] #number of features.

  Centroids=np.array([]).reshape(n,0)

  for i in range(K):
    rand=rd.randint(0,m-1)
    Centroids=np.c_[Centroids,df_ds_c[rand]]
  
  EuclidianDistance=np.array([]).reshape(m,0)

  for k in range(K):
    tempDist=np.sum((df_ds_c-Centroids[:,k])**2,axis=1)
    EuclidianDistance=np.c_[EuclidianDistance,tempDist]
  C=np.argmin(EuclidianDistance,axis=1)+1

  for i in range(no_of_iterations):
    
      EuclidianDistance=np.array([]).reshape(m,0)
      for k in range(K):
          tempDist=np.sum((df_ds_c-Centroids[:,k])**2,axis=1)
          EuclidianDistance=np.c_[EuclidianDistance,tempDist]
      C=np.argmin(EuclidianDistance,axis=1)+1
   
      Y={}
      for k in range(K):
          Y[k+1]=np.array([]).reshape(n,0)
      for i in range(m):
          Y[C[i]]=np.c_[Y[C[i]],df_ds_c[i]]
     
      for k in range(K):
          Y[k+1]=Y[k+1].T
          
      for k in range(K):
          Centroids[:,k]=np.mean(Y[k+1],axis=0)

      Output=Y
      for k in range(K):
        size.append(len(Output[k+1]))
  return (Output,Centroids,size)

In [62]:
kmeans = Kmeans_custom(features_df, 5, 50)
kmeans[0]

{1: array([[  1.,  39.,  69.,  91.],
        [  0.,  31.,  70.,  77.],
        [  1.,  40.,  71.,  95.],
        [  1.,  38.,  71.,  75.],
        [  1.,  39.,  71.,  75.],
        [  0.,  31.,  72.,  71.],
        [  0.,  29.,  73.,  88.],
        [  1.,  32.,  73.,  73.],
        [  0.,  35.,  74.,  72.],
        [  1.,  32.,  75.,  93.],
        [  0.,  32.,  76.,  87.],
        [  1.,  28.,  77.,  97.],
        [  0.,  32.,  77.,  74.],
        [  1.,  34.,  78.,  90.],
        [  1.,  39.,  78.,  88.],
        [  0.,  38.,  78.,  76.],
        [  0.,  27.,  78.,  89.],
        [  0.,  30.,  78.,  78.],
        [  0.,  30.,  78.,  73.],
        [  0.,  29.,  79.,  83.],
        [  0.,  31.,  81.,  93.],
        [  0.,  36.,  85.,  75.],
        [  0.,  33.,  86.,  95.],
        [  1.,  32.,  87.,  63.],
        [  1.,  28.,  87.,  75.],
        [  1.,  36.,  87.,  92.],
        [  0.,  30.,  88.,  86.],
        [  1.,  27.,  88.,  69.],
        [  1.,  35.,  93.,  90.],
        [  

In [49]:
np.c_?

Type:        CClass
String form: <numpy.lib.index_tricks.CClass object at 0x000002C57F9F5B38>
Length:      0
File:        c:\users\bansashi\anaconda3\envs\anaconda_3.6\lib\site-packages\numpy\lib\index_tricks.py
Docstring:  
Translates slice objects to concatenation along the second axis.

This is short-hand for ``np.r_['-1,2,0', index expression]``, which is
useful because of its common occurrence. In particular, arrays will be
stacked along their last axis after being upgraded to at least 2-D with
1's post-pended to the shape (column vectors made out of 1-D arrays).

See Also
--------
column_stack : Stack 1-D arrays as columns into a 2-D array.
r_ : For more detailed documentation.

Examples
--------
>>> np.c_[np.array([1,2,3]), np.array([4,5,6])]
array([[1, 4],
       [2, 5],
       [3, 6]])
>>> np.c_[np.array([[1,2,3]]), 0, 0, np.array([[4,5,6]])]
array([[1, 2, 3, ..., 4, 5, 6]])


In [39]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
def k_means(df):
    kmeans_kwargs = {
         "init": "k-means++",
         "n_init": 10,
         "max_iter": 100,
         "random_state": 42,
     }
    # A list holds the silhouette coefficients for each k
    silhouette_coefficients = []
    sse = []

    # Notice you start at 2 clusters for silhouette coefficient
    for k in range(2, 20):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(df)
        sse.append(kmeans.inertia_)
        score = silhouette_score(df, kmeans.labels_)
        silhouette_coefficients.append(score)
    return (kmeans, sse, silhouette_coefficients)

In [ ]:

pizza_customers_cluster = k_means(features_df[features])

In [ ]:
import matplotlib.pyplot as plt

figure, axes = plt.subplots(1, 1, figsize=(15,5))
axes.plot(range(2, 20), pizza_customers_cluster[1], label = "SSE")
axes.plot(range(2, 20), pizza_customers_cluster[2], label = "silhouette_coefficients")
axes.set_title("Clustering results")
axes.legend()

In [30]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(features_df[features])
clustered_df = input_df
clustered_df['cluster'] = kmeans.labels_

In [ ]:
clustered_df.to_csv("./cluster_results.csv")

### Cluster analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns



figure, axes = plt.subplots(1, 2, figsize=(20,8), subplot_kw={'ylim': (0,150)})

figure.tight_layout()

#clustered_df.drop("CustomerID", axis=1, inplace=True)
clustered_df[clustered_df['cluster'] == 0].boxplot(ax=axes[0])
clustered_df[clustered_df['cluster'] == 1].boxplot(ax=axes[1])

figure, axes = plt.subplots(1, 2, figsize=(20,8), subplot_kw={'ylim': (0,150)})
figure.tight_layout()
clustered_df[clustered_df['cluster'] == 2].boxplot(ax=axes[0])
clustered_df[clustered_df['cluster'] == 3].boxplot(ax=axes[1])

figure, axes = plt.subplots(1, 1, figsize=(20,8), subplot_kw={'ylim': (0,150)})
figure.tight_layout()

clustered_df[clustered_df['cluster'] == 4].boxplot(ax=axes)

plt.show()

* Cluster 1 -- Low income, low spend score, age range is 45 - 60 -- conservative family
* Cluster 2 -- High income, high spend, age range is 25 to 40 -- High spend niche customers
* Cluster 3 -- High income, Low spend, age 30 to 45 --- Economical family customers
* Cluster 4 -- Low income, high spend, age range 20 to 30 -- Fresher/ student
* Cluster 5 -- Medium income, medium spend, age range 25 to 50 -- Economical family